# Venues İstanbul
##  Problem Statement : 

İstanbul is most popular  city in Turkey and more than 15 million people choosing this beatifull city for their holiday every year from abroad. In this favourit holiday city, there are many many restaurants and cafes of different ambiance from many different food cultures. also there are many diffrent beachs resorts int this city.Traditional turkish cuisine, italian pizza restaurants, seafood restaurants, authentic middle east cafes, delicious kebab restaurants, turkish coffee, fast food, bistros, so many options for a tourist

-  -

##  Problem Discussion:
There is not enough time for a tourist who has been come  on vacation for 10 days to get the experience up to the right place to choose the right restaurant for dinner.there is no time for a tourist to learn which restaurant is cooking better or which restaurant is cooking more delicious fish or which bar is making better music

## Library Importing


In [1]:
# libraries for handle data and  data analsysis
import numpy as np 
import pandas as pd 
import json 

# handle requests libraries
import requests
from pandas.io.json import json_normalize

# Visiulation tools
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means 
from sklearn.cluster import KMeans



# map rendering library
!conda install -c conda-forge folium=0.5.0 --yes ## install folium
import folium 

# beautiful soup
from urllib.request import urlopen
from bs4 import BeautifulSoup

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

##


print('librariy importing is done !!')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

librariy importing is done !!


## Fetching data from fourSquare
We got ClientID and ClientSecret and we learned how we fetch data from fourSquare in earlier week

#### Getting Antalya geograpical coordinates with geopy

In [7]:
cntr = 'istanbul, Turkey'

geolocator = Nominatim()
location = geolocator.geocode(cntr)
latitude = location.latitude
longitude = location.longitude
print('Coordinates: {}, {}.'.format(latitude, longitude))

/home/savas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


Coordinates: 41.0766019, 29.052495.


#### Setting FourSquare Request

In [8]:
## Your Api ClientID adn ClientSecret
CLIENT_ID = '5ONQYHNWGBZ4O0RJPBCSSWV4R2JRSNRKNUNNBLJIY4D42DEI' 
CLIENT_SECRET = 'BUKLQNAH1XVPOKXOVB0N0BZFUWYAHMY44CKPXQFZUL54I43V'

# Foursquare API version
VERSION = '20200404' 


print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 1500 


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
     
    LIMIT)
print('Request Url:' + url)

CLIENT_ID: 5ONQYHNWGBZ4O0RJPBCSSWV4R2JRSNRKNUNNBLJIY4D42DEI
CLIENT_SECRET:BUKLQNAH1XVPOKXOVB0N0BZFUWYAHMY44CKPXQFZUL54I43V
Request Url:https://api.foursquare.com/v2/venues/explore?&client_id=5ONQYHNWGBZ4O0RJPBCSSWV4R2JRSNRKNUNNBLJIY4D42DEI&client_secret=BUKLQNAH1XVPOKXOVB0N0BZFUWYAHMY44CKPXQFZUL54I43V&v=20200404&ll=41.0766019,29.052495&limit=1500


#### Json data


In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9c26cf02a1720020ab80bb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 1058,
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 131,
  'suggestedBounds': {'ne': {'lat': 41.08609733788995,
    'lng': 29.06601277386834},
   'sw': {'lat': 41.067851961971584, 'lng': 29.03905803629286}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb8a8ef3db7b713ad75219a',
       'name': 'Bebek Koru Kahvesi',
       'location': {'address': 'Cevdet Paşa Cad. No:121/1 Bebek',
        'lat': 41.080080434213464,
        'lng': 29.051004483505697,
        'labeledLatLngs

#### How we can use this data ?
Using Geolocator one can get the latitude and Longitude values of a particular place after fetching these values we can use Forsquare API calls on these places and gather the venues that are currently happening around.
we can get venue categories from this data and  we can determine top venues for each category by like counts

### 1- Creating dataframe

In [11]:
def get_category(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
#Scraping data from the Foursquare API

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.id', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filtering category
nearby_venues['venue.categories'] = nearby_venues.apply(get_category, axis=1)

nearby_venues

/home/savas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,venue.name,venue.id,venue.categories,venue.location.lat,venue.location.lng
0,Bebek Koru Kahvesi,4bb8a8ef3db7b713ad75219a,Café,41.080080,29.051004
1,İoki,50377a92e4b0fc42f6c6bb79,Sushi Restaurant,41.073893,29.057801
2,Taps Bebek,4d20a974bdd7a0937537efce,Beer Bar,41.080102,29.050941
3,Boğaziçi Üniversitesi Bebek Havuzu,4c136c10a5eb76b070e1bfb7,Pool,41.080562,29.050485
4,Chilai,4cef9c5c13aea1438322749f,Lounge,41.078227,29.045544
...,...,...,...,...,...
95,Güney Yokuşu,50d3f83be4b044f16c8b8a9c,Scenic Lookout,41.083330,29.048457
96,küçüksu kasrı sarayı,55b4a4e7498e4c7fd0e7d3a0,Historic Site,41.078683,29.064788
97,Vaniköy Mutfak,5d0cd585deb4950025aeb560,Café,41.068985,29.057459
98,Niche İstanbul,5abe2ba3a35dce29f4fc7be9,Bistro,41.085268,29.053950


In [15]:
#fix the column names 

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,id,categories,lat,lng
0,Bebek Koru Kahvesi,4bb8a8ef3db7b713ad75219a,Café,41.080080,29.051004
1,İoki,50377a92e4b0fc42f6c6bb79,Sushi Restaurant,41.073893,29.057801
2,Taps Bebek,4d20a974bdd7a0937537efce,Beer Bar,41.080102,29.050941
3,Boğaziçi Üniversitesi Bebek Havuzu,4c136c10a5eb76b070e1bfb7,Pool,41.080562,29.050485
4,Chilai,4cef9c5c13aea1438322749f,Lounge,41.078227,29.045544
...,...,...,...,...,...
95,Güney Yokuşu,50d3f83be4b044f16c8b8a9c,Scenic Lookout,41.083330,29.048457
96,küçüksu kasrı sarayı,55b4a4e7498e4c7fd0e7d3a0,Historic Site,41.078683,29.064788
97,Vaniköy Mutfak,5d0cd585deb4950025aeb560,Café,41.068985,29.057459
98,Niche İstanbul,5abe2ba3a35dce29f4fc7be9,Bistro,41.085268,29.053950


In [16]:
nearby_venues['categories'].unique()

array(['Café', 'Sushi Restaurant', 'Beer Bar', 'Pool', 'Lounge', 'Beach',
       'Dessert Shop', 'Historic Site', 'Waterfront', 'Art Museum',
       'Ice Cream Shop', 'Coffee Shop', 'Hotel', 'Park', 'Movie Theater',
       'Steakhouse', 'Lighthouse', 'Pizza Place', 'Scenic Lookout',
       'Breakfast Spot', 'Kumpir Restaurant', 'Tea Room', 'Restaurant',
       'Theme Restaurant', 'Turkish Restaurant', 'BBQ Joint',
       'Seafood Restaurant', 'Burger Joint', 'Chocolate Shop', 'Bistro',
       'Doner Restaurant', 'Gym', 'Snack Place', 'Castle',
       'Sporting Goods Shop', 'Gym / Fitness Center', 'Music Venue',
       'Gourmet Shop', 'Art Gallery', 'Garden', 'Bar', 'Public Art',
       'Diner', 'Cocktail Bar', 'Speakeasy', 'Fishing Spot', 'Bakery',
       'Boutique'], dtype=object)